In [1]:
from pulp import *
# initialize the model
prob = LpProblem("aavinMilk", LpMaximize)

supply = {
    0 : 232031,
    1 : 249099,
    2 : 233415,
    3 : 217123,
    4 : 233866,
    5 : 265545,
    6 : 210676,
    7 : 225433,
    8 : 242297,
    9 : 262129,
    10: 218186,
    11: 223350,
    12: 249689,
    13: 232867,
    14: 249313,
    15: 256576,
    16: 267565,
    17: 271441,
    18: 277838,
    19: 250675,
    20: 250675,
    21: 241821,
    22: 258043,
    23: 246904,
    24: 231482,
    25: 249640,
    26: 256265,
    27: 208430,
    28: 254864,
    29: 220660
    }

demand = {
    0 : 286387,
    1 : 307453,
    2 : 288095,
    3 : 267987,
    4 : 288652,
    5 : 327752,
    6 : 260029,
    7 : 278243,
    8 : 299058,
    9 : 323536,
    10: 269299,
    11: 275672,
    12: 308182,
    13: 287419,
    14: 307718,
    15: 316682,
    16: 330245,
    17: 335029,
    18: 342925,
    19: 309399,
    20: 272493,
    21: 298470,
    22: 318493,
    23: 304744,
    24: 285709,
    25: 308121,
    26: 316298,
    27: 257257,
    28: 314569,
    29: 272352
    }
extra_reconstitution_cost = 2
butter_cost = 112
smp_cost = 163
other_supplier_cost = 16.5
normal_cost = 15
rmo_limit = 25000 # as per Sir

T = len(demand)
# ---------------------
# VARIABLES
# ---------------------

rm = LpVariable.dicts("rm", range(0, T), 0, None)
rmo = LpVariable.dicts("rmo", range(0, T), 0, None) 

smp = LpVariable.dicts("smp", range(0, T), 0, None)

butter = LpVariable.dicts("butter", range(0, T ), 0, None) 
isbutter = LpVariable.dicts("isbutter", range(0, T), 0, 1, cat=pulp.LpBinary)

cfwd = LpVariable.dicts("cfwd", range(0, T ), 0, None) 
cfsmp = LpVariable.dicts("cfsmp", range(0, T), 0, None)
cfbutter = LpVariable.dicts("cfbutter", range(0, T), 0, None)

output = LpVariable.dicts("output", range(0, T), 0, None)

usedbutter = LpVariable.dicts("usedbutter", range(0, T), 0, None)
usedsmp = LpVariable.dicts("usedsmp", range(0, T), 0, None)


# Constraints
prob += cfwd[0] == 12110
prob += cfsmp[0] == 1200
prob += cfbutter[0] == 520
prob += rm[0] == 0
prob += rmo[0] == 0
prob += smp[0] == 0
prob += butter[0] == 0
prob += output[0] == 0
prob += isbutter[0] == 0
prob += usedbutter[0] == 0
prob += usedsmp[0] == 0

for i in range(1, T):
    prob += cfwd[i-1] + .95*rm[i] + .95*rmo[i] + 10 * (usedsmp[i] + 2 * usedbutter[i]) == \
    output[i]
    prob += output[i] <= demand[i-1]
    prob += cfwd[i] == .05*rm[i] + .05*rmo[i]
    prob += rm[i] <= supply[i-1]
    
    prob +=  usedsmp[i] ==  2 * usedbutter[i]
    prob +=  cfbutter[i] <= 10000
    prob +=  cfbutter[i] <= 1000 * isbutter[i]
    
    
    prob += cfbutter[i] == cfbutter[i-1] + butter[i] - usedbutter[i]
    prob += cfsmp[i] == cfsmp[i-1] + smp[i] - usedsmp[i]
    prob += rmo[i] <= rmo_limit
  

# Objective function
prob += sum([22 * output[i] - 163 * smp[i] - 112 * butter[i] - 2500 * isbutter[i] - \
             19 * rm[i] - 20.5 * rmo[i] - \
             21 * 10 * (usedsmp[i] + 2 * usedbutter[i]) for i in range(1,T)]), "Objective"
    
    
prob.writeLP("aavinMilk.lp")
    
status = prob.solve(GLPK(options=["--ranges","aavinMilk.sen"]))
print(status)

#print the result
print("rm")
for i in range(1, T):
    print(rm[i].value())


print("rmo")       
for i in range(1, T):
    print(rmo[i].value())

print("isbutter")   
for i in range(1, T):
    print(isbutter[i].value())

    
print("butter")   
for i in range(1, T):
    print(butter[i].value())
    
print("smp")    
for i in range(1, T):
    print(smp[i].value())

print("cfw")       
for i in range(1, T):
    print(cfwd[i].value())

print("cfbutter")   
for i in range(1, T):
    print(cfbutter[i].value())
    
print("cfsmp")    
for i in range(1, T):
    print(cfsmp[i].value())

    
print("usedsmp")    
for i in range(1, T):
    print(usedsmp[i].value())

print("usedbutter")    
for i in range(1, T):
    print(usedbutter[i].value())
    
print("Objective", value(prob.objective))

1
rm
232031.0
249099.0
233415.0
217123.0
233866.0
265545.0
210676.0
225433.0
242297.0
262129.0
218186.0
223350.0
249689.0
232867.0
249313.0
256576.0
267565.0
271441.0
277838.0
250675.0
250675.0
241821.0
258043.0
246904.0
231482.0
249640.0
256265.0
208430.0
254864.0
rmo
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
21650.5
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
25000.0
isbutter
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
butter
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
smp
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
cfw
12851.6
13705.0
12920.8
12106.2
12943.3
14527.2
11783.8
12521.7
13364.9
14356.5
12159.3
12417.5
13734.5
12893.4
13715.7
14078.8
14628.2
14822.1
15141.9
13783.8
13616.3
13341.1
14152.2


In [25]:
#%load aavinMilk.sen
